In [ ]:
# Install required packages
!pip install numpy pandas scikit-learn nltk scikit-surprise

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [45 lines of output]
      Compiling surprise/similarities.pyx because it changed.
      Compiling surprise/prediction_algorithms/matrix_factorization.pyx because it changed.
      Compiling surprise/prediction_algorithms/optimize_baselines.pyx because it changed.
      Compiling surprise/prediction_algorithms/slope_one.pyx because it changed.
      Compiling surprise/prediction_algorithms/co_clustering.pyx because it changed.
      [1/5] Cythonizing surprise/prediction_algorithms/co_clustering.pyx
      
      Error compiling Cython file:
      ------------------------------------------------------------
      ...
              self.avg_cltr_i = avg_cltr_i
              self.avg_cocltr = avg_cocltr
      
              return self
      
          def compute_averages(self, np.ndarray[np.int_t] cltr_u,
                                                   ^
  


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


# Text Vectorization
Convert article text into numerical vectors using TF-IDF vectorization from scikit-learn.

In [3]:
# Download dataset
# If you already have the dataset, you can skip or comment this step
!kaggle datasets download -d hsankesara/medium-articles

In [ ]:
# Load and prepare data
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Load dataset
df = pd.read_csv('articles.csv')

# Display the first few rows and columns 
print("Available columns:", df.columns.tolist())
print("\nSample data:")
print(df.head())

Available columns: ['author', 'claps', 'reading_time', 'link', 'title', 'text']

Sample data:
             author claps  reading_time  \
0        Justin Lee  8.3K            11   
1       Conor Dewey  1.4K             7   
2  William Koehrsen  2.8K            11   
3      Gant Laborde  1.3K             7   
4  Emmanuel Ameisen   935            11   

                                                link  \
0  https://medium.com/swlh/chatbots-were-the-next...   
1  https://towardsdatascience.com/python-for-data...   
2  https://towardsdatascience.com/automated-featu...   
3  https://medium.freecodecamp.org/machine-learni...   
4  https://blog.insightdatascience.com/reinforcem...   

                                               title  \
0  Chatbots were the next big thing: what happene...   
1  Python for Data Science: 8 Concepts You May Ha...   
2  Automated Feature Engineering in Python – Towa...   
3  Machine Learning: how to go from Zero to Hero ...   
4  Reinforcement Learning from

In [3]:
# Clean Data Based on Available Columns

# In case you use a different data set, you can replace the list below with the actual columns from your dataset
# For example, if 'url' is missing, exclude it, etc.
available_columns = ['title', 'text', 'claps', 'author', 'reading_time', 'link'] 

# Check if additional columns exist before selecting
required_columns = ['author', 'text', 'claps', 'title', 'reading_time', 'link']
existing_columns = [col for col in required_columns if col in df.columns]

# Select only the existing columns
df = df[existing_columns]

# Display the cleaned dataFrame
print("Columns used for recommendation:", df.columns.tolist())
print("\nCleaned data sample:")
print(df.head())

Columns used for recommendation: ['author', 'text', 'claps', 'title', 'reading_time', 'link']

Cleaned data sample:
             author                                               text claps  \
0        Justin Lee  Oh, how the headlines blared:\nChatbots were T...  8.3K   
1       Conor Dewey  If you’ve ever found yourself looking up the s...  1.4K   
2  William Koehrsen  Machine learning is increasingly moving from h...  2.8K   
3      Gant Laborde  If your understanding of A.I. and Machine Lear...  1.3K   
4  Emmanuel Ameisen  Want to learn about applied Artificial Intelli...   935   

                                               title  reading_time  \
0  Chatbots were the next big thing: what happene...            11   
1  Python for Data Science: 8 Concepts You May Ha...             7   
2  Automated Feature Engineering in Python – Towa...            11   
3  Machine Learning: how to go from Zero to Hero ...             7   
4  Reinforcement Learning from scratch – Insight ... 

In [4]:
# Topic-based recommendation function
def get_recommendations(topic, n=2):
    
    # Match by tags
    mask = df['tags'].str.contains(topic, case=False, na=False)
    matches = df[mask].sort_values('claps', ascending=False)

    if len(matches) >= n:
        return matches.head(n)[['title', 'publication', 'reading_time', 'tags']]

    # If not enough matches, use content similarity
    tfidf = TfidfVectorizer(stop_words='english')
    text_matrix = tfidf.fit_transform(df['text'])
    query_vec = tfidf.transform([topic])
    similarities = cosine_similarity(query_vec, text_matrix)

    top_indices = similarities[0].argsort()[-n:][::-1]
    return df.iloc[top_indices][['title', 'publication', 'reading_time', 'tags']]

In [5]:
# Interactive interface
def interactive_recommendations():
    topic = input("What topic interests you? ")
    print("\nFinding best articles about", topic, "...")
    recommendations = get_recommendations(topic)

    print("\nRecommended articles:")
    for idx, row in recommendations.iterrows():
        print(f"\nTitle: {row['title']}")
        print(f"Publication: {row['publication']}")
        print(f"Reading time: {row['reading_time']} minutes")
        print(f"Tags: {row['tags']}")

In [ ]:
#  Run system
interactive_recommendations()


Finding best articles about data ...


KeyError: 'tags'